In [4]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import pickle
import os

# Define the search term variable and login credentials
email = "andrei13@gmail.com"
password = "aqDfiR7@o!&"
walmart_products_excel_filename="walmart_products.xlsx"

# Set a timeout duration for the wait
timeout = 30  # seconds

# Define the list of food items
#foods_list = ['TORTILLA DE HARINA','FRIJOLES','HUEVO','QUESO','LECHUGA','MELON','TOSTADAS HORNEADAS','SALSA MEXICANA','POLLO PECHUGA','FRIJOLES','PAPAYA','HUEVO','JAMON DE PAVO','SALSA MEXICANA','TORTILLA DE MAIZ','MELON','LECHE','PLATANO','ALMENDRAS','YOGURT','MANZANA','LECHE','FRESAS','ALMENDRAS','MOLIDA DE RES','ZANAHORIA','ESPINACAS','PASTA','AGUACATE','POLLO PECHUGA','SQUASH','CHAYOTE','ZANAHORIA','TORTILLA DE MAIZ','SALMON','JITOMATE','PEPINO','GARBANZOS','ARROZ','AGUACATE','NARANJA','CACAHUATES','UVAS','ALMENDRAS','PAPAYA','NUEZ','ATUN','TOSTADAS HORNEADAS','JITOMATE','PAN INTEGRAL','QUESO','JAMON','PEPINO','CREMA','TORTILLA DE MAIZ','QUESO OAXACA','NOPALES']
foods_list = ['chicken thighs']

# Iterate over each element in the list and replace spaces with a plus sign '+'
foods_list_with_plus = [food.replace(" ", "+") for food in foods_list]

options = uc.ChromeOptions()
options.headless = False  # Set headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = uc.Chrome(options=options)

# Construct the URL for the Walmart login page
home_page_url = "https://www.walmart.com/account/login"

# Path to save or load the cookies
cookies_path = 'cookies.pkl'

# Function to load cookies if they exist
def load_cookies(driver, path):
    if os.path.exists(path):
        with open(path, 'rb') as cookiesfile:
            cookies = pickle.load(cookiesfile)
            for cookie in cookies:
                driver.add_cookie(cookie)

# Function to save cookies
def save_cookies(driver, path):
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

# Function to determine the tier based on 'special_notes'
def determine_tier(note):
    if 'Bought' in note:
        return 'Tier 1'
    elif 'Best' in note:
        return 'Tier 2'
    elif 'Popular' in note:
        return 'Tier 3'
    elif note == '':
        return 'Tier 5'
    else:
        return 'Tier 4'
    
try:
    # Navigate to the login page
    driver.get(home_page_url)
    print(f"Navigated to {home_page_url}")
    
    # Load cookies and refresh the page to use them
    load_cookies(driver, cookies_path)
    driver.refresh()
    print(f"Loaded cookies")
    
    try:
        time.sleep(1)
                
        # Looking for the 'Sign in text'
        element = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/section/header/h1')
    
        if element.text == 'Sign in or create your account':
            print(f"Need to enter credentials, cookie {cookies_path} didn't work")

            # Wait until the element is present
            element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="react-aria-1"]'))
            WebDriverWait(driver, timeout).until(element_present)

            # Locate the email input box and enter the email address
            email_input = driver.find_element(By.XPATH, '//*[@id="react-aria-1"]')
            email_input.send_keys(email)
            print("Typed email")

            # Locate the continue button and click it
            continue_button = driver.find_element(By.XPATH, '//*[@id="login-continue-button"]')
            continue_button.click()
            print("Clicked continue")

            # Wait until the element is present
            element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="react-aria-4"]'))
            WebDriverWait(driver, timeout).until(element_present)

            # type the password
            password_input = driver.find_element(By.XPATH, '//*[@id="react-aria-4"]')
            password_input.send_keys(password)
            print("Typed password")

            sign_in_button = driver.find_element(By.XPATH, '//*[@id="withpassword-sign-in-button"]')
            sign_in_button.click()
            print("Clicked Sign in")
        
    except NoSuchElementException:
        print("Label Sign in doesn't exist")
        
    print("Navigating to homepage")
    
    # Wait until the element is present
    element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/span/header/form/div/input'))
    WebDriverWait(driver, timeout).until(element_present)
    
    print("Made it to homepage")
    
    # Save the cookies
    with open(cookies_path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)
    
    # Initialize an empty list to hold all product details
    products = []
        
    for food in foods_list_with_plus:
        # Search term
        #food = "tortillas+de+harina"
        # Construct the URL with the search term variable
        url = f"https://www.walmart.com/search?q={food}"
        print(f"Searching for {food}")

        # Navigate to the page
        driver.get(url)

        # Wait until the element is present
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="maincontent"]/main/div/div[2]/div/div/div[1]/div[2]/div[1]/section/div/div[1]/div/div/a'))
        WebDriverWait(driver, timeout).until(element_present)      

        # Loop through the product listings
        for i in range(1, 5):  # Adjust the range as needed
            try:
                print(f"Getting object {i}")

                # Construct XPaths for the product name, image, and special notes
                name_xpath = f'//*[@id="maincontent"]/main/div/div[2]/div/div/div[1]/div[2]/div[1]/section/div/div[{i}]/div/div/div/div[2]/span/span'
                img_xpath = f'//*[@id="maincontent"]/main/div/div[2]/div/div/div[1]/div[2]/div[1]/section/div/div[{i}]/div/div/div/div[1]/div[2]/div[1]/img'
                special_notes_xpath = f'//*[@id="maincontent"]/main/div/div[2]/div/div/div[1]/div[2]/div[1]/section/div/div[{i}]/div/div/div/div[1]/div[1]/span'
                url_xpath = f'//*[@id="maincontent"]/main/div/div[2]/div/div/div[1]/div[2]/div[1]/section/div/div[{i}]/div/div/a'

                # Extract product details
                product_name = driver.find_element(By.XPATH, name_xpath).text
                product_img = driver.find_element(By.XPATH, img_xpath).get_attribute('src')
                try:
                    product_special_notes = driver.find_element(By.XPATH, special_notes_xpath).text
                except NoSuchElementException:
                    #print("No special notes for this product")
                    product_special_notes=""
                product_url = driver.find_element(By.XPATH, url_xpath).get_attribute('href')

                orig_food_name=food.replace("+", " ")
                # Store product details in a dictionary
                product_details = {
                    'orig_food_name': orig_food_name,
                    'name': product_name,
                    'special_notes': product_special_notes,
                    'img_url': product_img,
                    'url': product_url
                }

                # Append the product details dictionary to the products list
                products.append(product_details)

            except NoSuchElementException as e:
                #print(f"Object {i} doesn't exist, skipping")
                pass
    
except Exception as e:
    print(f"An Exception occurred : {str(e)}")

    
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(products)

# Apply the function to create a new 'Tier' column
df['Tier'] = df['special_notes'].apply(determine_tier)

# Sorting by 'orig_food_name' in ascending order and 'special_notes' in descending order
df_sorted = df.sort_values(by=['orig_food_name','Tier','special_notes'], ascending=[True, True,False])

# Picking the first element of each 'orig_food_name'
first_elements = df_sorted.groupby('orig_food_name').head(1).reset_index(drop=True)

# Export the DataFrame to an Excel file
first_elements.to_excel(walmart_products_excel_filename, index=False)

print("End of script")

# Quit the driver
driver.quit()


Navigated to https://www.walmart.com/account/login
Loaded cookies
Need to enter credentials, cookie cookies.pkl didn't work
Typed email
Clicked continue
Typed password
Clicked Sign in
Navigating to homepage
An Exception occurred : Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.107)
Stacktrace:
	GetHandleVerifier [0x00384CA3+225091]
	(No symbol) [0x002B4DF1]
	(No symbol) [0x00159A7A]
	(No symbol) [0x0013E312]
	(No symbol) [0x001B517B]
	(No symbol) [0x001C55A6]
	(No symbol) [0x001AF2F6]
	(No symbol) [0x001879B9]
	(No symbol) [0x0018879D]
	sqlite3_dbdata_init [0x007F9A43+4064547]
	sqlite3_dbdata_init [0x0080104A+4094762]
	sqlite3_dbdata_init [0x007FB948+4072488]
	sqlite3_dbdata_init [0x004FC9A9+930953]
	(No symbol) [0x002C07C4]
	(No symbol) [0x002BACE8]
	(No symbol) [0x002BAE11]
	(No symbol) [0x002ACA80]
	BaseThreadInitThunk [0x766A7BA9+25]
	RtlInitializeExceptionChain [0x7760BE3B+107]
	RtlClearBits [0x7760BD

NameError: name 'products' is not defined

In [11]:
 
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(products)

# Apply the function to create a new 'Tier' column
df['Tier'] = df['special_notes'].apply(determine_tier)

# Sorting by 'orig_food_name' in ascending order and 'special_notes' in descending order
df_sorted = df.sort_values(by=['orig_food_name','Tier','special_notes'], ascending=[True, True,False])

,food_name,name,special_notes,img_url,url,Tier
0,chicken thighs,Freshness Guaranteed Boneless Skinless Chicken...,Bought 1 time,https://i5.walmartimages.com/asr/8d322aa9-8df3...,https://www.walmart.com/ip/Freshness-Guarantee...,Tier 1
1,chicken thighs,Freshness Guaranteed Boneless Skinless Chicken...,Popular pick,https://i5.walmartimages.com/asr/40ee61a6-4db2...,https://www.walmart.com/ip/Freshness-Guarantee...,Tier 3
2,chicken thighs,"Perdue Harvestland, Boneless Skinless Chicken ...",Best seller,https://i5.walmartimages.com/asr/cdb95bc3-5294...,https://www.walmart.com/ip/Perdue-Harvestland-...,Tier 2
3,chicken thighs,"Perdue, No Antibiotics Ever, Skin-on Chicken T...",,https://i5.walmartimages.com/asr/51bc2d7c-3c93...,https://www.walmart.com/ip/Perdue-No-Antibioti...,Tier 5


In [13]:
import re
from fractions import Fraction

def to_float(s):
    try:
        # First, try to convert directly to a float
        return float(s)
    except ValueError:
        try:
            # If ValueError is raised, try to interpret as a Fraction
            return float(Fraction(s))
        except ValueError:
            # If it still raises a ValueError, it might be an empty string or None
            if s == "" or s is None:
                # You can define what a default return value should be for empty strings
                return 0.0
            else:
                # If string is not empty or None, it's an invalid format
                raise ValueError(f"Could not convert string to float: '{s}'")
                
# Construct the URL for the Walmart login page
home_page_url = "https://www.walmart.com/account/login"

options = uc.ChromeOptions()
options.headless = False  # Set headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = uc.Chrome(options=options)

# Navigate to the login page
driver.get(home_page_url)
print(f"Navigated to {home_page_url}")
    
load_cookies(driver, cookies_path)
driver.refresh()
print(f"Loaded cookies")

walmart_products_excel_filename="walmart_products.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(walmart_products_excel_filename)
print("Loaded Excel")

df['Calories'] = ""
df['Fat'] = ""
df['Carbs'] = ""
df['Protein'] = ""
df['serving_size'] = ""
df['serving_size_num1'] = ""
df['serving_size_measurement1'] = ""
df['serving_size_num2'] = ""
df['serving_size_measurement2'] = ""
df['Serving_per_container'] = ""

# Iterate over the values in the "url" column
for index, row in df.iterrows():

    try:
        # Navigate to each page
        current_url=row['url']
        current_food=row['food_name']
        driver.get(current_url)
        print(f"Checking {current_food} {current_url}")

        # Wait until the element is present
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[2]'))
        WebDriverWait(driver, timeout).until(element_present)
        
        # Initiliaze the number to empty
        fat_num=0
        carbs_num=0
        protein_num=0
        serving_size=""
        serving_size_num1=0
        serving_size_measurement1=""
        serving_size_num2=0
        serving_size_measurement2=""
        calories_num=0
        servings_per_container=""
        
        # Get Calories details
        try:
            # Construct XPaths for the calories name and the calories number
            calories_name_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/div[4]/span[1]'
            calories_num_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/div[4]/span[2]'
            
            servings_per_container_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/div[2]/p[1]'
            
            serving_size_name_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/div[2]/p[2]/span[1]'
            serving_size_num_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/div[2]/p[2]/span[2]'
            
            # find the elements
            calories_name = driver.find_element(By.XPATH, calories_name_xpath).text
            calories_num = driver.find_element(By.XPATH, calories_num_xpath).text
            
            servings_per_container_num = driver.find_element(By.XPATH, servings_per_container_xpath).text
            servings_per_container_num = servings_per_container_num.split("serving")[0].strip()
            
            serving_size_name = driver.find_element(By.XPATH, serving_size_name_xpath).text
            serving_size_num = driver.find_element(By.XPATH, serving_size_num_xpath).text
            
            # Regular expression pattern to match a number followed by optional whitespace and a unit of measurement
            #pattern = r"(\d+(\.\d+)?)(\s*)([a-zA-Z]+)"
            pattern = r"(\d+\/\d+|\d+(\.\d+)?)(\s*)([a-zA-Z]+(?:\s+[a-zA-Z]+)?)"
            
            # Split if there is another measurement between ()
            serving_size_num1=serving_size_num.split("(")[0].strip()
            
            # Divide the text from the number
            match = re.match(pattern, serving_size_num1)
            if match:
                serving_size_num1 = match.group(1)
                serving_size_measurement1 = match.group(4)
            
            # If there is a second part, get the text in between the ()
            try:
                serving_size_num2=serving_size_num.split("(")[1].strip().split(")")[0].strip()
                
                # Divide the text from the number
                match = re.match(pattern, serving_size_num2)
                if match:
                    serving_size_num2 = match.group(1)
                    serving_size_measurement2 = match.group(4)
            except Exception as e:
                pass
                #print("There wasn't another measurement")
            
            # Update the df with the number found
            df.at[index, 'Calories'] = to_float(calories_num)
            
            df.at[index, 'serving_size'] = serving_size_num
            df.at[index, 'serving_size_num1'] = to_float(serving_size_num1)
            df.at[index, 'serving_size_measurement1'] = serving_size_measurement1
            df.at[index, 'serving_size_num2'] = to_float(serving_size_num2)
            df.at[index, 'serving_size_measurement2'] = serving_size_measurement2
            
            df.at[index, 'Serving_per_container'] = servings_per_container_num
            
        except NoSuchElementException as e:
            print(f"Missing element found for this product str{e}")
        
        # Find all elements that match the given XPath
        elements = driver.find_elements(By.XPATH, '//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/table/tbody[1]/tr')
        
        # Get the number of elements in the table
        number_of_elements = len(elements)
        #print(f"number_of_elements: {number_of_elements}")
        
        # Get the rest of the elements details
        for i in range(1, number_of_elements + 1):
            try:
                # Construct XPaths for each element
                element_name_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/table/tbody[1]/tr[{i}]/td[1]/span[1]'
                element_num_xpath = f'//*[@id="maincontent"]/section/main/div[2]/div[2]/div/div[1]/div/div/section[*]/section/div[2]/div/div/section/table/tbody[1]/tr[{i}]/td[1]/span[2]'
                
                element_name = driver.find_element(By.XPATH, element_name_xpath).text
                element_num = driver.find_element(By.XPATH, element_num_xpath).text
                pattern = r"g.*$|G.*$"
                element_num = re.sub(pattern, '', element_num)
                #element_num=element_num.split("g")[0].strip()
                
                if element_name == 'Total Fat':
                    df.at[index, 'Fat'] = to_float(element_num)
                elif element_name == 'Total Carbohydrate':
                    df.at[index, 'Carbs'] = to_float(element_num)
                elif element_name == 'Protein':
                    df.at[index, 'Protein'] = to_float(element_num)
                
            except NoSuchElementException:
                print(f"{i} element does not exist in this table")
        
        
        #print(print(df.loc[index]))
        
    except Exception as e:
        print(f"Some error ocurred: {e}")
        
# Export the DataFrame to an Excel file
df.to_excel(walmart_products_excel_filename, index=False)

Navigated to https://www.walmart.com/account/login
Loaded cookies
Loaded Excel
Some error ocurred: 'food_name'


In [3]:
import undetected_chromedriver.v2 as uc
driver = uc.Chrome()
driver.get('https://www.example.com')


ModuleNotFoundError: No module named 'undetected_chromedriver.v2'